In [1]:
import pandas as pd
import numpy as np


import fitz
from camel_tools.utils.dediac import dediac_ar
from ar_corrector.corrector import Corrector
from deep_translator import GoogleTranslator

from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords

from summarizer import Summarizer,TransformerSummarizer
import spacy
from sklearn.cluster import DBSCAN
from sklearn.svm import SVC

from gensim.models import word2vec
from gensim.models.doc2vec import TaggedDocument
from gensim.parsing.preprocessing import strip_tags
from gensim.utils import simple_preprocess

from lbl2vec import Lbl2Vec


2022-10-14 10:55:02.658890: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-14 10:55:02.659046: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-14 10:55:06.188633: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-10-14 10:55:06.188888: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-14 10:55:06.189006: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcub

In [2]:
path = '/home/noahdarwich/code/NoahDarwich/pdf_coder/raw_data/20194004.1.pdf'
with fitz.open(path) as doc:
    for page in doc:
        blocks = page.get_text_blocks()
        print(blocks)

[(280.7419128417969, 248.9887237548828, 497.44732666015625, 278.641845703125, ' ءانجسلا نيب ةاواسملاو ةلادعلا\n', 0, 0), (420.81500244140625, 269.98870849609375, 497.466064453125, 299.641845703125, 'نييسايسلا\n', 1, 0), (222.86676025390625, 1506.174072265625, 912.664306640625, 1517.2939453125, '• لاير 4 رطق • لاير 4 ةيدوعسلا • هينج 2 نادوسلا • مهارد 3 تاراملإا • سلف 100 تيوكلا • ةريل 1000 نانبل • هينج 2 رصم • مهرد 5 برغملا • رانيد 1 رئازجلا • ناموت 5 ناريإ • تاريل 10 ايروس • سلف 100 ندرلأا • رانيد 500 :قارعلا :ةديرجلا راعسأ\nPrice: Europe € 1\n', 2, 0), (918.0, 1491.82568359375, 977.31494140625, 1514.868896484375, '<image: Indexed(176,ICCBased(RGB,sRGB IEC61966-2.1)), width: 124, height: 49, bpc: 8>', 3, 1), (83.65819549560547, 337.5774841308594, 210.86415100097656, 377.1774597167969, 'يس�ايس�لا بتكلما حيرس�ت\nيقارعلا يعويس�لا بزحلل\n', 4, 0), (49.45280075073242, 380.657470703125, 245.0720977783203, 426.2574768066406, 'ةييرهاملجا تاجاجتحلاا\nداس�فلاو ةس�س�احلما ماظنل راذنا س�رج\n', 5, 

In [3]:
def convert_pdf_to_text(path):
    with fitz.open(path) as doc:
        for page in doc:
            blocks = page.get_text_blocks()
            texts = []
            for block in blocks:
                text = block[4:-2][0]
                text = dediac_ar(text[::-1]).replace('�','').strip('\n').replace('\n','')
                texts.append(text)
    corr = Corrector()
    corrected = [corr.contextual_correct(x) for x in texts]
    return corrected


In [4]:
def translate_texts(text):
    translator = GoogleTranslator(source='ar', target='en')
    translated = translator.translate(text)

    return translated

In [5]:
def filtering_events(texts):
    
    ps = PorterStemmer()
    key_words = ['occupation','sit-in','demonstration','protest','roadblock','blockade','attack','march','strike']
    key_words = [ps.stem(word) for word in key_words]
    
    # texts = [word_tokenize(text) for text in texts]
    
    events = []
    not_events = []
    for text in texts:
        if any(x in [ps.stem(word) for word in word_tokenize(text)] for x in key_words):
            events.append(text)
        else:
            not_events.append(text) 
        
    return events, not_events

In [6]:
path1 = '/home/noahdarwich/code/NoahDarwich/pdf_coder/raw_data/20194004.1.pdf'
texts1 = convert_pdf_to_text(path1)
texts1 = [x for x in texts1 if len(x)>100]
texts1 = [x.split('.') for x in texts1]

In [7]:
path2 = '/home/noahdarwich/code/NoahDarwich/pdf_coder/raw_data/20194116.1.pdf'
texts2 = convert_pdf_to_text(path2)
texts2 = [x for x in texts2 if len(x)>100]
texts2 = [x.split('.') for x in texts2]

In [11]:
texts1

[['و ما eporuE :ecirPأسعار الجديدة العراق 005 دينار في المدن 001 فلس من سوريا في ليرات و إيران في يوما من الجزائر في دينار في المغرب في درهم في مصر في جنيه في لبنان 0001 ليرة في الكويت 001 فلس من الإمارات في دراهم في السودان في جنيه في السعودية في ريال في قطر في ريال في'],
 ['الذكر الطيب على الدوام للشهداء والشفاء العاجل للجرحى ',
  ' بدولة المواطنة والديمقراطية والعدالة الاجتماعية ',
  ' نظام الحاشدة والفساد وتحقيق البديل الوطني المتمثل والعلمية للمواطنين وفتح فضاءات للتغيير الذي وانهاء اننا جدد انحياز حزبنا الكامل على جانب المطالب العادلة والعلاقة ',
  ' جانبها يف تصحيح الشارات وتوفير الحياة الكريمة والمنة والمشارك للجماهير يف مطالبها العادلة ويبقى يقف دوما إلى أن ان حزبنا كان ويبقى على موقفه التضامن والداعم مناقشة التطورات الراهنة يف جلسة طارئة لمجلس النواب العام للقوات المسلحة والوزراء ورؤساء الأجهزة الأمنية في ان من الدوري استدعاء رئيس مجلس الوزراء والقائد الاحتجاجية للمتظاهرين واملحتجني، ما يوجب الحفاظ على خلفية الحركة المختلفة وعلى الممتلكات العامة والشخصية هو مصدر قوة في ان احل

In [8]:
stop_words = stopwords.words('english')
stop_words.remove('against')

In [9]:
texts_sep = []
for text in texts1:
    for i in range(len(text)):
        if len(text[i]) > 10:
          texts_sep.append(text[i])
texts_sep = [translate_texts(text) for text in texts_sep]

In [12]:
texts_sep_2 = []
for text in texts2:
    for i in range(len(text)):
        if len(text[i]) > 10:
          texts_sep_2.append(text[i])
texts_sep_2 = [translate_texts(text) for text in texts_sep_2]

In [13]:
cleaned_2 = []
for sen in (texts_sep_2):
    for word in stop_words:
        sen = sen.replace(" "+ word +" "," ")
    cleaned_2.append(sen)

In [14]:
cleaned = []
for sen in (texts_sep):
    for word in stop_words:
        sen = sen.replace(" "+ word +" "," ")
    cleaned.append(sen)

In [15]:
cleaned = [sent_tokenize(sen) for sen in cleaned]

In [16]:
cleaned_2 = [sent_tokenize(sen) for sen in cleaned_2]

In [17]:
cleaned_2

[['And eporuE: ecirP New prices Iraq 005 dinars cities 001 fils Syria liras Iran day From Algeria dinars Morocco dirhams Egypt pounds Lebanon 0001 lira Kuwait 001 fils Emirates dirhams Sudan pounds Saudi Arabia riyals Qatar riyals in'],
 ['On review electoral system importance cooperation United Nations, Dennis Balsakhrat, people meeting, United Nations Iraq, Janice, received Special Representative, Secretary-General'],
 ['The National Cannes, please amend Italian list, upcoming Dialogue project,” reviewing “the commission’s preparations conducting elections, importance security logistical support, president stressed statement Republic.”'],
 ['It hoped final version elections completed soon special team, completed governorates, minority, Electoral Commission.'],
 ['Protecting peaceful protesters'],
 ['For elections, sources stressed need independent supreme commandment work provide assistance image address conduct free fair elections.',
  'Security support The challenge efforts aimed, 

In [18]:
final_extracted = []
for sen in cleaned:
    if len(sen) > 1:
        for i in range(len(sen)):
            final_extracted.append(sen[i])
    else:
        final_extracted.append(sen[0])

In [19]:
len(final_extracted)

95

In [20]:
test_ex = []
for sen in cleaned_2:
    if len(sen) > 1:
        for i in range(len(sen)):
            test_ex.append(sen[i])
    else:
        test_ex.append(sen[0])

In [21]:
df = pd.read_csv('../raw_data/tishreen_full.csv')
train_df = df[df['URL1'] == 'https://tareeqashaab.com/images/TariqPDF/2019/10/3.pdf']
target_df = [translate_texts(text) for text in list(train_df['narrative'])]

In [88]:
df.shape

(2641, 103)

In [25]:
targets = []
for sen in target_df:
    for word in stop_words:
        sen = sen.replace(" "+ word +" "," ")
    targets.append(sen)

In [26]:
len(targets)

19

In [27]:
nlp = spacy.load('en_core_web_lg')
sim_dict = {}
for i in range(len(final_extracted)):
    for j in range(len(targets)):
        X = nlp(final_extracted[i])
        y = nlp(targets[j])
        val = X.similarity(y)
        sim_dict[val] = [targets[j],final_extracted[i]]

In [41]:
sim_df = pd.DataFrame(sim_dict).T.reset_index()
sim_df.columns = ['similarity','target', 'extracted']

In [42]:
sim_df.sort_values(by=['similarity'],ascending=False, inplace=True)
high_sim = sim_df[sim_df['similarity']>0.8]

In [43]:
high_sim.reset_index(drop=True, inplace=True)
high_sim.head(20)

,similarity,target,extracted
0,0.971135,Hundreds demonstrated Karbala front governorat...,"which raised Diyala, demonstrators circulated ..."
1,0.970506,Demonstrations front Diyala Provincial Council...,Corruption money management circles job opport...
2,0.953893,Demonstrations Wasit near provincial council b...,Demonstrations took place governorate support ...
3,0.951420,Al-Haboubi Square demonstration Dhi Qar govern...,"Services protection corrupt, city Hilla, deman..."
4,0.941251,A demonstration front Maysan Provincial Counci...,"The flag Iraq condemned corruption science, pa..."
5,0.937998,Al-Haboubi Square demonstration Dhi Qar govern...,While cut streets Karkh cities angry protests ...
6,0.935415,A demonstration front Maysan Provincial Counci...,Discussing current developments emergency sess...
7,0.933436,Thousands Babil Governorate people gathered fr...,"Services protection corrupt, city Hilla, deman..."
8,0.931515,"Angry citizens demonstrated Najaf province, ag...","which raised Diyala, demonstrators circulated ..."
9,0.930751,Al-Haboubi Square demonstration Dhi Qar govern...,"The guard directed demonstrators, firing tear ..."


In [44]:
print(f"Number of similar sentences: {len(high_sim)}")
print('-------------')
print(f"Acual events vs Potential events: {len(targets)} / {len(final_extracted)}")

Number of similar sentences: 427
-------------
Acual events vs Potential events: 19 / 95


In [45]:
print(high_sim.extracted[2])
print('-------------')
print(high_sim.target[2])

Demonstrations took place governorate support group, demonstrations near council building, demonstrations took place city
-------------
Demonstrations Wasit near provincial council building support series demonstrations took place provinces


In [46]:
from collections import Counter

full_text = ' '.join(targets)
tags = Counter(full_text.split()).most_common(40)
print(tags)

[('security', 11), ('forces', 11), ('Square', 10), ('Tahrir', 9), ('demonstrators', 8), ('Angry', 8), ('demonstration', 7), ('protests', 7), ('due', 7), ('cutting', 7), ('roads', 7), ('leading', 7), ('area', 6), ('governorate', 6), ('A', 5), ('corruption', 5), ('\u200b\u200bBaghdad', 5), ('front', 5), ('building', 4), ('burned', 4), ('council', 4), ('against', 4), ('Provincial', 4), ('building,', 3), ('job', 3), ('Demonstrations', 3), ('provincial', 3), ('calling', 2), ('unemployment,', 2), ('Baghdad', 2), ('Dhi', 2), ('Qar', 2), ('demanding', 2), ('services', 2), ('opportunities,', 2), ('support', 2), ('demonstrated', 2), ('government', 2), ('demand', 2), ('protest', 2)]


In [47]:
full_text_2 = ' '.join(final_extracted)
tags_2 = Counter(full_text_2.split()).most_common(50)
print(tags_2)

[('The', 32), ('security', 14), ('people', 13), ('forces', 12), ('demonstrators', 11), ('demonstrations', 11), ('Iraqi', 10), ('demands', 9), ('right', 9), ('government', 8), ('governorate', 8), ('services', 8), ('Baghdad', 7), ('said', 7), ('council', 7), ('And', 6), ('current', 6), ('rights', 6), ('failure', 6), ('protests', 6), ('statement', 6), ('front', 6), ('Iraq', 5), ('cities', 5), ('wounded', 5), ('demonstration', 5), ('violence', 5), ('We', 5), ('corruption', 5), ('reality', 5), ('job', 5), ('yesterday', 5), ('building', 5), ('citizens', 4), ('peaceful', 4), ('ministers', 4), ('protest', 4), ('public', 4), ('against', 4), ('demanded', 4), ('use', 4), ('water', 4), ('demonstrators,', 4), ('place', 4), ('legitimate', 4), ('state', 4), ('human', 4), ('work', 4), ('opportunities', 4), ('people,', 4)]


In [48]:
all_tags = {'event':('protest march sit-in occupation roadblock blockade strike demonstrators demand'),
 'not_event':('services statement said concern use legitimate state place council')}

In [49]:
ps = PorterStemmer()
key_words = ['occupation','sit-in','demonstration','protest','roadblock','blockade','attack','march','strike']
key_words = [ps.stem(word) for word in key_words]
key_words = {key:i+1 for i,key in enumerate(key_words)}
key_words_df = pd.DataFrame(all_tags,index=[0]).T.reset_index()
key_words_df.columns = ['labels','keywords']
key_words_df['label_indx'] = [1,0]
key_words_df

,labels,keywords,label_indx
0,event,protest march sit-in occupation roadblock bloc...,1
1,not_event,services statement said concern use legitimate...,0


In [50]:
def tokenize(doc):
    return simple_preprocess(strip_tags(doc), deacc=True, min_len=2, max_len=15)

key_words_df['keywords'] = key_words_df['keywords'].apply(lambda x: x.split(' '))
key_words_df['keywords'] = key_words_df['keywords'].apply(lambda description_keywords: [keyword.lower() for keyword in description_keywords])

In [54]:
key_words_df

,labels,keywords,label_indx
0,event,"[protest, march, sit-in, occupation, roadblock...",1
1,not_event,"[services, statement, said, concern, use, legi...",0


In [51]:
sim_df['tagged_docs'] = sim_df.apply(lambda row: TaggedDocument(tokenize(row['extracted']), [str(row.name)]), axis=1)

In [55]:
key_words_df['tagged_key'] = key_words_df['keywords'].apply(ps.stem(word) for word in key_words_df['keywords'])

In [57]:
Lbl2Vec_model = Lbl2Vec(keywords_list=list(key_words_df.keywords), 
                        tagged_documents=sim_df.tagged_docs, 
                        label_names=list(key_words_df.labels),
                        min_count=1, 
                        similarity_threshold=0.70, 
                        min_num_docs=10, 
                        epochs=20)
Lbl2Vec_model.fit()

2022-10-14 13:43:07,123 - Lbl2Vec - INFO - Train document and word embeddings
2022-10-14 13:43:12,288 - Lbl2Vec - INFO - Train label embeddings
2022-10-14 13:43:12,290 - Lbl2Vec - WARNING - The following keywords from the 'keywords_list' are unknown to the Doc2Vec model and therefore not used to train the model: march sit-in occupation roadblock blockade strike


In [58]:
test_df = pd.DataFrame(test_ex, columns=['test_sen'])
test_df['tagged_test'] = test_df.apply(lambda row: TaggedDocument(tokenize(row['test_sen']), [str(row.name)]), axis=1)
test_df

,test_sen,tagged_test
0,And eporuE: ecirP New prices Iraq 005 dinars c...,"([and, eporue, ecirp, new, prices, iraq, dinar..."
1,On review electoral system importance cooperat...,"([on, review, electoral, system, importance, c..."
2,"The National Cannes, please amend Italian list...","([the, national, cannes, please, amend, italia..."
3,It hoped final version elections completed soo...,"([it, hoped, final, version, elections, comple..."
4,Protecting peaceful protesters,"([protecting, peaceful, protesters], [4])"
...,...,...
65,"Najaf strangers without interruption, same, an...","([najaf, strangers, without, interruption, sam..."
66,"There nothing new time activists, buses well, ...","([there, nothing, new, time, activists, buses,..."
67,"The end forty-day visit, talk resounding disob...","([the, end, forty, day, visit, talk, resoundin..."
68,The protests renewed state religious disobedie...,"([the, protests, renewed, state, religious, di..."


In [59]:
result_df = Lbl2Vec_model.predict_new_docs(test_df.tagged_test)
result_df

2022-10-14 13:44:16,203 - Lbl2Vec - INFO - Calculate document embeddings
2022-10-14 13:44:16,247 - Lbl2Vec - INFO - Calculate document<->label similarities


,doc_key,most_similar_label,highest_similarity_score,event,not_event
0,0,event,0.993284,0.993284,0.992942
1,1,not_event,0.816183,0.812173,0.816183
2,2,event,0.844301,0.844301,0.840885
3,3,event,0.901056,0.901056,0.898064
4,4,event,0.263797,0.263797,0.261709
...,...,...,...,...,...
65,65,event,0.990960,0.990960,0.990494
66,66,not_event,0.889505,0.887954,0.889505
67,67,event,0.875510,0.875510,0.872182
68,68,event,0.947229,0.947229,0.945538


In [60]:
result_df.sort_values(by=['highest_similarity_score'],ascending=False, inplace=True)
result_df

,doc_key,most_similar_label,highest_similarity_score,event,not_event
32,32,event,0.995554,0.995554,0.995185
5,5,not_event,0.995507,0.995002,0.995507
45,45,event,0.995269,0.995269,0.994965
24,24,event,0.994236,0.994236,0.994124
0,0,event,0.993284,0.993284,0.992942
...,...,...,...,...,...
14,14,not_event,0.139127,0.132945,0.139127
34,34,event,0.072188,0.072188,0.067799
37,37,event,-0.006230,-0.006230,-0.006440
10,10,event,-0.055171,-0.055171,-0.055308


In [85]:
len(result_df[result_df['most_similar_label']=='event'].doc_key)

38

In [83]:
key_lst = [int(x) for x in result_df[result_df['most_similar_label']=='not_event'].doc_key]

In [84]:
for i in key_lst:
    print(test_df.iloc[i].test_sen)
    print('-----------------------')

For elections, sources stressed need independent supreme commandment work provide assistance image address conduct free fair elections.
-----------------------
There killing destruction, whoever thinks oppression, injustice violence raise performance official institutions, “training work fortifying blood fighting legitimate corruption street creating chance make progress terms demand positions necessary peaceful reforms.”
-----------------------
The legitimate demands protesters real sports economic field originally met.”
-----------------------
It provocative act redouble efforts ground, stressed parties must intermittent internet.
-----------------------
In case laws related electoral system, Barham negotiating representative United Nations Population, expressing concern situation calling protection demonstrators
-----------------------
Moreno singled reform would increase violence exacerbate fragile situation image peaceful protests would peaceful Iraqi order exercise restraint secu

In [10]:
# path1 = '/home/noahdarwich/code/NoahDarwich/pdf_coder/raw_data/20194004.1.pdf'
# path2 = '/home/noahdarwich/code/NoahDarwich/pdf_coder/raw_data/20193011_1.pdf'

# texts1 = convert_pdf_to_text(path1)
# texts2 = convert_pdf_to_text(path2)

# all_texts1 = [translate_texts(text) for text in texts1]
# all_texts2 = [translate_texts(text) for text in texts2]

In [28]:
# cleaned_sens = []
# for sen,sen2 in zip(all_texts1,all_texts2):
#     for word in stop_words:
#         sen = sen.replace(" "+ word +" "," ")
#         sen2 = sen2.replace(" "+ word +" "," ")

#     cleaned_sens.append(sen)
#     cleaned_sens.append(sen2)

'Its side correcting badges, providing decent peaceful life, participating demands masses, always standing party maintains solidarity supportive stance. Discussing current developments emergency session General Assembly Armed Forces, ministers heads security services, necessary summon Prime Minister Commander The protest demonstrators protesters, requires preserving background various movement public personal property source strength dreams against background demonstration scientific protest forms fell demonstrators martyrs people Iraq'

In [ ]:
# long_sents =[]
# for sen in cleaned_sens:
#     if len(sen.split()) > 3:
#         long_sents.append(sen)

In [ ]:
# targets = [sent_tokenize(text) for text in targets]

In [ ]:
# all_data = targets + sens

In [ ]:
# nlp = spacy.load('en_core_web_lg')

# sent_vecs = {}
# docs = []

# for sen in all_data:
#     doc = nlp(sen[0])
#     docs.append(doc)
#     sent_vecs.update({sen[0]:doc.vector})

In [ ]:
# sentences = list(sent_vecs.keys())
# vectors = list(sent_vecs.values())

In [ ]:
# X = np.array(vectors)
 
# dbscan = DBSCAN(eps=0.01, min_samples=10).fit(X)

In [ ]:
# dbscan.labels_

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1])

In [ ]:
# results = pd.DataFrame({'label':dbscan.labels_, 'sent':sentences})
# results

In [ ]:
# vocab = [x[0].split() for x in all_data]

In [ ]:
# w2v_model  = word2vec.Word2Vec(sentences=vocab,
#                                vector_size=100)

# voc_vec = w2v_model.build_vocab(vocab,
#                        progress_per=10000,
#                        update=True)
#  word2vec.Word2Vec(all_data, min_count=1)

# w2v_model.build_vocab(w2v_model)

In [ ]:
# w2v_model.train(vectors, 
#                 total_examples=w2v_model.corpus_count, epochs=10)

In [ ]:
# w2v_model.raw_vocab.keys()

In [ ]:
# svm_model = SVC()
# svm_model.fit(X)

In [ ]:
# from gensim.models import KeyedVectors
# filename = 'GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin'

# g_model = KeyedVectors.load_word2vec_format(filename, binary=True)

In [ ]:
# result = g_model.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)
# result


In [ ]:
# g_model.add_vectors(keys=sent_vecs.keys(), weights=vectors)

In [ ]:
# model = TransformerSummarizer(transformer_type="XLNet",transformer_model_key="xlnet-base-cased")
# summary_2 = [model(text, min_length=3) for text in not_events]
# summary_2